In [ ]:
import cv2
import numpy as np
from PIL import Image
from os import listdir
import matplotlib.pyplot as plt
import skimage.transform as sktr

In [ ]:
# Read in the image and converts to [0,1] scale
def prep_img(path):
    im = cv2.imread(path, 0)
    im = im.astype(float) / 255
    return get_color_channels(im)

# Separate the image into rgb color channels
def get_color_channels(im):
    height = np.floor(im.shape[0] / 3.0).astype(np.int32)
    b = im[:height]
    g = im[height: 2*height]
    r = im[2*height: 3*height]
    return r, g, b

# Display the image in notebook
def show_img(img):
    plt.imshow(img)
    plt.show()
    
# Saves the give images as jpg files
def save_img(imname, result):
    result = (result * 255).astype(np.uint8)
    result = Image.fromarray(result, 'RGB')
    imname = imname.split('.')[0]
    result.save(f'./colorized_imgs/{imname}.jpg')

In [ ]:
# Shifts image by x_off and y_off
def shift_img(img, x_off, y_off):
    shifted = np.roll(img, x_off, axis=1)
    shifted = np.roll(shifted, y_off, axis=0)
    return shifted

# Crop image by given percent on each dimension
def crop(img, percent):
    x = int(img.shape[0] * percent / 2)
    y = int(img.shape[1] * percent / 2)
    return img[x:img.shape[0]-x,y:img.shape[1]-y]

# Returns the SSD btwn two images
def score(img1, img2):
    return np.sum((img1-img2)**2)

In [ ]:
# Uses naive window search to align two images
def align(img1, img2):
    offsets = list(range(-15, 15))
    
    # Intialize score, offset, and best image
    min_score = float("inf")
    best_off = [0,0]
    best_img = None
     
    # Crop images to remove noise
    img1 = crop(img1, 0.2)
    img2 = crop(img2, 0.2)
    
    # Iterate through all possible offset combos
    for x_off in offsets:
        for y_off in offsets:
            # Shift the image by current offset
            shifted = shift_img(img1, x_off, y_off)
            
            # Calculate current similarity score 
            curr_score = score(shifted, img2)
            
            # Update best offset if score improves
            if curr_score <= min_score:
                min_score = curr_score
                best_off = [x_off, y_off]
                best_img = shifted
    
    return best_img, best_off           

In [ ]:
# Uses pyramid method to align two images
def align_pyramid(pyr1, pyr2):
    best_off, new_off = [0,0], [0,0]
    for img1, img2 in zip(pyr1, pyr2):
        # Mutiply best offset by 2 since image scales up by 2
        best_off = np.multiply(best_off, 2)
        
        # Shift the current image by current best offset
        img1 = shift_img(img1, best_off[0], best_off[1])

        # Align the new image
        _, new_off = align(img1, img2)
        
        # Add in new offset to total offset
        best_off[0] += new_off[0]
        best_off[1] += new_off[1]
    
    # Shift the original image by best offset
    final_img = shift_img(pyr1[len(pyr1)-1], best_off[0], best_off[1])
    return final_img, best_off

In [ ]:
# Creates an image pyramid by halving the image resolution at each subsqequent level
def generate_pyramid(img, levels=5):
    img_pyramid = [img]
    
    for i in range(1,levels+1):
        prev_img = img_pyramid[i-1]
        
        x_dim = int(prev_img.shape[1] * 0.5)
        y_dim = int(prev_img.shape[0] * 0.5)
        dim = (x_dim, y_dim)
    
        img_pyramid.append(cv2.resize(prev_img, dim))
        
    return list(reversed(img_pyramid))

In [ ]:
# Preps, aligns, and overlays the rgb channels to create colorized image using naive search method.
def colorize_naive(imname):
    input_path = f'./data/{imname}'
    r,g,b = prep_img(input_path)

    # Align the r and g channels to the b channel
    ar, r_off = align(r, b)
    ag, g_off = align(g, b)
    ab, b_off = crop(b, 0.2), [0,0]

    # Print the channel offsets
    print(f'Red channel offset: {r_off[::-1]}')
    print(f'Green channel offset: {g_off[::-1]}')
    print(f'Blue channel offset: {b_off[::-1]}')

    # Create a color image
    im_out = np.dstack([ar, ag, ab])
    
    return im_out

In [ ]:
# Preps, aligns, and overlays the rgb channels to create colorized image using pyramid method.
def colorize_pyramid(imname):
    print(f"Curr image: {imname}")
    input_path = f'./data/{imname}'
    r,g,b = prep_img(input_path)

    # Create image pyramid for each color channel
    r_pyramid = generate_pyramid(r)
    g_pyramid = generate_pyramid(g)
    b_pyramid = generate_pyramid(b)

    # Align the r and g channels to the b channel
    print("Aligning Red Channel")
    ar, r_off = align_pyramid(r_pyramid, b_pyramid)
    
    print("Aligning Green Channel")
    ag, g_off = align_pyramid(g_pyramid, b_pyramid)
    
    print("Aligning Blue Channel")
    ab, b_off = b, [0,0]

    # Print the channel offsets
    print(f'Red channel offset: {r_off[::-1]}')
    print(f'Green channel offset: {g_off[::-1]}')
    print(f'Blue channel offset: {b_off[::-1]}')
    
    # Create a color image
    im_out = np.dstack([ar, ag, ab])
    
    # Crop border
    result = crop(im_out, 0.1)

    return result

In [ ]:
# Create lists of both the small and large images
small_imgs = []
large_imgs = []
for file in listdir('./data'):
    if file.endswith('.jpg'):
        small_imgs.append(file)
    elif file.endswith('.tif'):
        large_imgs.append(file)  

In [ ]:
# Driver Code for small images
for imname in small_imgs:
    # Get the aligned rgb image
    result = colorize_naive(imname)    
    # Store the image
    save_img(imname, result)
    # Diplay the image
    show_img(result)

In [ ]:
# Driver Code for large images
for imname in large_imgs:
    # Get the aligned rgb image
    result = colorize_pyramid(imname)    
    # Store the image
    save_img(imname, result)
    # Diplay the image
    show_img(result)

In [ ]:
# Extra Credit

# Rescale image intensities to incroproate more wide range of colors.
# Uses cumaltive sum to calcualte the CDF of each pixel value, then use it to map to a value
def auto_contrast(img):
    x, counts = np.unique(img, return_counts=True)
    cusum = np.cumsum(counts)
    pixel_mapping = {x[i]: cusum[i] / cusum[-1] * 255 for i in range(len(x))}
    convert = np.vectorize(lambda x: pixel_mapping[x])
    return convert(img).astype(np.uint8)

def auto_crop(img):
    print

In [ ]:
# Driver Code for contrasting images
for imname in small_imgs:
    im = colorize_naive(imname)
    contrast = auto_contrast(im)
    # Display the images
    show_img(im)
    show_img(contrast)